Create the tables for this section.

In [1]:
%load_ext sql       
# Connect to an empty SQLite database
%sql sqlite://

'Connected: None@None'

In [2]:
%%sql
DROP TABLE IF EXISTS Purchase;
-- Create tables
CREATE TABLE Purchase (
    Product  VARCHAR(255), 
    Date     DATE, 
    Price    FLOAT,
    Quantity INT
);

-- Insert tuples
INSERT INTO Purchase VALUES ('Bagel', '10/21', 1, 20);
INSERT INTO Purchase VALUES ('Bagel', '10/25', 1.5, 20);
INSERT INTO Purchase VALUES ('Banana', '10/3', 0.5, 10);
INSERT INTO Purchase VALUES ('Banana', '10/10', 1, 10);

SELECT * FROM Purchase;

Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


[('Bagel', '10/21', 1.0, 20),
 ('Bagel', '10/25', 1.5, 20),
 ('Banana', '10/3', 0.5, 10),
 ('Banana', '10/10', 1.0, 10)]

## Aggregation Operations

SQL support several __aggregation__ operations
* SUM, COUNT, MIN, MAX, AVG
* Except COUNT, all aggregations apply to a signle attribute

### COUNT

Syntax
```mysql
SELECT COUNT(column_name)
FROM   table_name
WHERE  condition;
```
  
> __Example:__ Find the number of purchases

| Product | Date  | Price | Quantity |
|---------|-------|-------|----------|
| Bagel   | 10/21 | 1     | 20       |
| Bagel   | 10/25 | 1.5   | 20       |
| Banana  | 10/3  | 0.5   | 10       |
| Banana  | 10/10 | 1     | 10       |

In [3]:
%%sql
SELECT COUNT(Product)
FROM   Purchase;

Done.


[(4,)]

* Count applies to duplicates, unless otherwise stated
  * Same as ```COUNT(*)```. Why?
  
> __Example:__ Find the number of different product purchases

* Use DISTINCT

In [4]:
%%sql
SELECT COUNT(DISTINCT Product)
FROM   Purchase;

Done.


[(2,)]

### SUM

Syntax
```mysql
SELECT SUM(column_name)
FROM   table_name
WHERE  condition;
```

> __Example:__ How many units of all products have been purchased?

| Product | Date  | Price | Quantity |
|---------|-------|-------|----------|
| Bagel   | 10/21 | 1     | 20       |
| Bagel   | 10/25 | 1.5   | 20       |
| Banana  | 10/3  | 0.5   | 10       |
| Banana  | 10/10 | 1     | 10       |

In [5]:
%%sql
SELECT SUM(Quantity)
FROM   Purchase;

Done.


[(60,)]

> __Example:__ How many Bagels have been purchased?

In [6]:
%%sql
SELECT SUM(Quantity)
FROM   Purchase
WHERE  Product = 'Bagel'

Done.


[(40,)]

## AVG

Syntax
```mysql
SELECT AVG(column_name)
FROM   table_name
WHERE  condition;
```

> __Example:__ What is the average sell price of Bagels?

| Product | Date  | Price | Quantity |
|---------|-------|-------|----------|
| Bagel   | 10/21 | 1     | 20       |
| Bagel   | 10/25 | 1.5   | 20       |
| Banana  | 10/3  | 0.5   | 10       |
| Banana  | 10/10 | 1     | 10       |

In [7]:
%%sql
SELECT AVG(Price)
FROM   Purchase
WHERE  Product = 'Bagel';

Done.


[(1.25,)]

### Simple Aggregations

> __Example:__ Total earnings from Bagels sold?

In [8]:
%%sql
SELECT SUM(Price * Quantity)
FROM   Purchase
WHERE  Product = 'Bagel';

Done.


[(50.0,)]

## GROUP BY

Ued with aggregate functions (COUNT, MAX, MIN, SUM, AVG) to group the result-set by one or more columns.

Syntax
```mysql
SELECT   column_name(s)
FROM     table_name
WHERE    condition
GROUP BY column_name(s)
[ORDER BY column_name(s)];
```

> __Example:__ Find total sales after 10/1 per product

| Product | Date  | Price | Quantity |
|---------|-------|-------|----------|
| Bagel   | 10/21 | 1     | 20       |
| Bagel   | 10/25 | 1.5   | 20       |
| Banana  | 10/3  | 0.5   | 10       |
| Banana  | 10/10 | 1     | 10       |

In [9]:
%%sql
SELECT   Product, SUM(price * quantity) AS TotalSales
FROM     Purchase
WHERE    Date > '10/1'
GROUP BY Product;

Done.


[('Bagel', 50.0), ('Banana', 15.0)]

### Grouping and Aggregation: Semantics of the Query

__1.__ Compute the FROM and WHERE clauses

In [10]:
%%sql
SELECT *
FROM   Purchase
WHERE  Date > '10/1'

Done.


[('Bagel', '10/21', 1.0, 20),
 ('Bagel', '10/25', 1.5, 20),
 ('Banana', '10/3', 0.5, 10),
 ('Banana', '10/10', 1.0, 10)]

__2.__ Group attributes according to GROUP BY

|__Product__| __Date__ |__Price__|__Quantity__|
|:-------:|:--------:|:-----:|:--------:|
|  Bagel  | 10/21/17 |   1   |    20    |
|         | 10/25/17 |  1.5  |    20    |
|  Banana | 10/03/17 |  0.5  |    10    |
|         | 10/10/17 |   1   |    10    |


__Caution:__ SQL _only_ displays one row if no aggregation function is used

In [11]:
%%sql
SELECT   *
FROM     Purchase
WHERE    Date > '10/1'
GROUP BY Product;

Done.


[('Bagel', '10/25', 1.5, 20), ('Banana', '10/10', 1.0, 10)]

In [12]:
%%sql
SELECT   Product, Count(Product)
FROM     Purchase
WHERE    Date > '10/1'
GROUP BY Product;

Done.


[('Bagel', 2), ('Banana', 2)]

__3.__ Compute the SELECT clause: grouped attributes and aggregates

In [13]:
%%sql  -- Find total sales after '10/1' per product

SELECT   Product, SUM(price * quantity) AS TotalSales
FROM     Purchase
WHERE    Date > '10/1'
GROUP BY Product;

Done.


[('Bagel', 50.0), ('Banana', 15.0)]

### GROUP BY vs Nested Queries

In [14]:
%%sql
SELECT DISTINCT x.Product, (SELECT Sum(y.price*y.quantity)
                                   FROM Purchase y
                                   WHERE x.product = y.product
                                         AND y.date > '10/1') AS TotalSales
FROM  Purchase x
WHERE x.date > '10/1';

Done.


[('Bagel', 50.0), ('Banana', 15.0)]

## HAVING

* HAVING clauses contains conditions on __aggregates__
* WHERE clauses condition on __individual tuples__

Syntax
```mysql
SELECT   column_name(s)
FROM     table_name
WHERE    condition
GROUP BY column_name(s)
HAVING   condition
[ORDER BY column_name(s)];
```

> __Example:__ Same query as before, except that we consider only products with more than 30 units sold

In [15]:
%%sql
SELECT   Product, SUM(price * quantity) AS TotalSales
FROM     Purchase
WHERE    Date > '10/1'
GROUP BY Product
HAVING SUM(Quantity) > 30;

Done.


[('Bagel', 50.0)]

# Advanced\* Topics

In this section
* Relational Division is SQL
* Nulls (revisited)
* Outer Joins

## Relational Division in SQL

* Not supported as a primitive operator, but useful for expressing queries like:

> _"Find suppliers who sell the x parts..."_

> _"Find buyers who bught all products from a given category..."_

* Let $A$ have 2 fields, $x$ and $y$, $B$ have only field $y$
```mysql
A(x, y)
B(y)
```
  * $A/B$ contains all $x$ tuples such that for every $y$ tuple in $B$, there is an $xy$ tuple in $A$
  * Or: If the set of $y$ values associated with an $x$ value in $A$ contains all $y$ values in $B$, the $x$ value is in $A/B$.

Classic Option 1
```mysql
%%sql
SELECT T1.x
FROM A AS T1
WHERE NOT EXISTS( SELECT T2.y
                  FROM B AS T2
                  EXCEPT
                  SELECT T3.y
                  FROM A AS T3
                  WHERE T3.y=T1.y);
```

Classic Option 2 (without EXCEPT)
```mysql
%%sql
SELECT DISTINCT T1.x
FROM A AS T1
WHERE NOT EXISTS(SELECT T2.y
                 FROM B AS T2
                 WHERE NOT EXISTS (SELECT T3.x
                                   FROM A AS T3
                                   WHERE T3.x=T1.x
                                   AND T3.y=T2.y
                                  )
                );
```

In [16]:
%%sql
DROP TABLE IF EXISTS A;
-- Create tables
CREATE TABLE A (
    x VARCHAR,
    y VARCHAR);

DROP TABLE IF EXISTS B1;
-- Create tables
CREATE TABLE B1 (
    y VARCHAR);

DROP TABLE IF EXISTS B2;
-- Create tables
CREATE TABLE B2 (
    y VARCHAR);

DROP TABLE IF EXISTS B3;
-- Create tables
CREATE TABLE B3 (
    y VARCHAR);

-- Insert tuples
INSERT INTO A VALUES ('x1', 'y1');
INSERT INTO A VALUES ('x1', 'y2');
INSERT INTO A VALUES ('x1', 'y3');
INSERT INTO A VALUES ('x1', 'y4');
INSERT INTO A VALUES ('x2', 'y1');
INSERT INTO A VALUES ('x2', 'y2');
INSERT INTO A VALUES ('x3', 'y2');
INSERT INTO A VALUES ('x4', 'y2');
INSERT INTO A VALUES ('x4', 'y4');
INSERT INTO B1 VALUES ('y2');
INSERT INTO B2 VALUES ('y2');
INSERT INTO B2 VALUES ('y4');
INSERT INTO B3 VALUES ('y1');
INSERT INTO B3 VALUES ('y2');
INSERT INTO B3 VALUES ('y4');

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [17]:
%%sql
SELECT * FROM A;

Done.


[('x1', 'y1'),
 ('x1', 'y2'),
 ('x1', 'y3'),
 ('x1', 'y4'),
 ('x2', 'y1'),
 ('x2', 'y2'),
 ('x3', 'y2'),
 ('x4', 'y2'),
 ('x4', 'y4')]

In [18]:
%%sql
SELECT * FROM B1;

Done.


[('y2',)]

In [19]:
%%sql  -- Change bellow to perform: A/B1, A/B2, A/B3
SELECT DISTINCT T1.x
FROM A AS T1
WHERE NOT EXISTS(SELECT T2.y
                 FROM B1 AS T2
                 WHERE NOT EXISTS (SELECT T3.x
                                   FROM A AS T3
                                   WHERE T3.x=T1.x
                                   AND T3.y=T2.y
                                  )
                );

Done.


[('x1',), ('x2',), ('x3',), ('x4',)]

### Yet another option

[“A Simpler (and Better) SQL Approach to Relational Division”](https://users.dcc.uchile.cl/~cgutierr/cursos/BD/divisionSQL.pdf)

Journal of Information Systems Education, Vol. 13(2)

## Null Values

* For _numerical operations_, NULL -> NULL:
  * If x is NULL then ```4*(3-x)/7``` is still NULL

* For _boolean operations_, in SQL there are three values:

```
FALSE  = 0
UNKNOWN = 0.5
TRUE    = 1
```

* If x is NULL then ```x = “Joe”``` is UNKNOWN

```
C1 AND C2   = min(C1, C2)
C1 OR C2     = max(C1, C2)
NOT C1       = 1 – C1
```

>  __Example:__
```mysql
SELECT *
FROM   Person
WHERE  (age < 25)
  AND  (height > 6 AND weight > 190);
```
Won’t return: 
age=20
height=NULL
weight=200

__Rule in SQL:__ include only tuples that yield TRUE (1.0)

> __Example:__ Unexpected behavior
```mysql
SELECT *
FROM   Person
WHERE  age < 25 OR age >= 25;
```
Some tuples from _Person_ are not included

Test for NULL expliitly:
* x IS NULL
* x IS NOT NULL


>```mysql
SELECT *
FROM   Person
WHERE  age < 25 OR age >= 25 OR age IS NULL;
```
Now it includes all tuples in _Person_

## Inner Joins + NULLS = Lost data?

* By default, joins in SQL are __inner joins__

```
Product(name, category)
Purchase(prodName, store)
```

Syntax 1
```mysql
SELECT Product.name, Purchase.store
FROM   Product
JOIN   Purchase ON Product.name = Purchase.prodName;
```

Syntax 2
```mysql
SELECT Product.name, Purchase.store
FROM   Product, Purchase
WHERE  Product.name = Purchase.prodName;
```

* Both equivalent, both _inner joins_

* __However:__ Products that never sold (with no Purchase tuple) will be lost!

## Outer Joins
* An __outer join__ returns tuples from the joined relations that don’t have a corresponding tuple in the other relations
  * i.e. If we join relations A and B on a.X = b.X, and there is an entry in A with X=5, but none in B with X=5
  LEFT [OUTER] JOIN will return a tuple __(a, NULL)__
 
Syntax
```mysql
SELECT column_name(s)
FROM   table1
LEFT OUTER JOIN table2 ON table1.column_name = table2.column_name;
```

In [20]:
%%sql

-- Create tables
DROP TABLE IF EXISTS Product;
CREATE TABLE Product (
    name VARCHAR(255) PRIMARY KEY,
    category VARCHAR(255)
);

DROP TABLE IF EXISTS Purchase;
CREATE TABLE Purchase(
    prodName varchar(255),
    store varchar(255)
);

-- Insert tuples
INSERT INTO Product VALUES ('Gizmo', 'Gadget');
INSERT INTO Product VALUES ('Camera', 'Photo');
INSERT INTO Product VALUES ('OneClick', 'Photo');

INSERT INTO Purchase VALUES ('Gizmo', 'Wiz');
INSERT INTO Purchase VALUES ('Camera', 'Ritz');
INSERT INTO Purchase VALUES ('Camera', 'Wiz');

Done.
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [21]:
%%sql
SELECT *
FROM   Product;

Done.


[('Gizmo', 'Gadget'), ('Camera', 'Photo'), ('OneClick', 'Photo')]

In [22]:
%%sql
SELECT *
FROM   Purchase;

Done.


[('Gizmo', 'Wiz'), ('Camera', 'Ritz'), ('Camera', 'Wiz')]

In [23]:
%%sql
SELECT Product.name, Purchase.store
FROM   Product
INNER JOIN Purchase
ON Product.name = Purchase.prodName;

Done.


[('Gizmo', 'Wiz'), ('Camera', 'Ritz'), ('Camera', 'Wiz')]

## Outer Joins
* __Left outer join__
  * Include the left tuple even if there is no match
* __Right outer join__
  * Include the right tuple even if there is no match
* __Full outer join__
  * Include both left and right tuples even if there is no match

# Summary

* The relational model has rigorously defined query languages that are simple and powerful.
* Several ways of expressing a given query; a query optimizer should choose the most efficient version.
* SQL is the lingua franca (common language) for accessing relational database systems.
* SQL is a rich language that handles the way data is processed ___declaratively___
  * Expresses the logic of a computation without describing its control flow

___

In [24]:
# Modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom.css").read()
    return HTML(styles)
css_styling()